# Depedencies

In [1]:
import pandas as pd
import numpy as np
import time
import collections
import plotly.graph_objects as go
import plotly.express as px
import math
import seaborn as sn
import datetime

# Import the Data from the excel

In [2]:
# Read the printify products excel file into pandas dataframe
orders = pd.read_excel('Printify_Data_Analyst_homework.xlsx', sheet_name = 'Orders')
orders.head()

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,TOTAL_SHIPPING,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT
0,7143578,2013244.178,2013244,US,WA,2020-10-02 19:30:15,2020-10-20 07:53:06,NaN,9,2902,2200,2020-05-19 15:22:57,True,business_account_3,UPS,2020-10-08 21:07:29
1,5888534,1358429.695,1358429,US,MN,2020-11-21 05:19:03,2020-11-23 00:23:32,NaN,4,1149,924,2018-02-08 07:34:02,True,business_account_3,USPS,2020-11-28 21:50:56
2,6484224,939390.193,939390,US,AK,2020-10-25 01:39:08,2020-10-26 20:30:53,NaN,1,977,400,2019-04-09 20:02:46,True,business_account_3,USPS,2020-11-02 23:37:54
3,7251033,1856107.400,1856107,US,PA,2020-10-08 16:35:21,2020-10-10 15:02:47,NaN,1,661,800,2020-07-20 17:04:41,True,business_account_3,UPS,2020-10-15 18:08:37
4,7241488,1843904.400,1843904,US,CO,2020-11-17 19:30:10,2020-11-18 03:45:25,NaN,2,719,424,2020-07-15 18:47:51,True,business_account_3,USPS,2020-11-24 00:44:56


In [3]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13506 entries, 0 to 13505
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   MERCHANT_ID             13506 non-null  int64         
 1   ORDER_ID                13506 non-null  float64       
 2   SHOP_ID                 13506 non-null  int64         
 3   ADDRESS_TO_COUNTRY      13505 non-null  object        
 4   ADDRESS_TO_REGION       12930 non-null  object        
 5   ORDER_DT                13506 non-null  datetime64[ns]
 6   FULFILLED_DT            12637 non-null  datetime64[ns]
 7   REPRINT_FLAG            103 non-null    float64       
 8   SALES_CHANNEL_TYPE_ID   13506 non-null  int64         
 9   TOTAL_COST              13506 non-null  int64         
 10  TOTAL_SHIPPING          13506 non-null  int64         
 11  MERCHANT_REGISTERED_DT  13506 non-null  datetime64[ns]
 12  SUB_IS_ACTIVE_FLAG      13506 non-null  bool  

In [4]:
# Read the printify products excel file into pandas dataframe
new_line_items = pd.read_excel('Printify_Data_Analyst_homework.xlsx', sheet_name = 'Line items')
new_line_items.head(20)

,ORDER_ID,PRINT_PROVIDER_ID,PRODUCT_BRAND,PRODUCT_TYPE,ITEM_STATUS,QUANTITY
0,1.750015e+06,30,Generic brand,Mug,on-hold,3
1,1.750015e+06,30,Generic brand,Mug,on-hold,2
2,1.225155e+06,16,Gildan,Kids clothes,shipment_delivered,1
3,1.828891e+06,39,Bella+Canvas,T-Shirt,shipment_delivered,1
4,1.847848e+06,10,Generic brand,Accessories,shipment_delivered,1
5,1.750016e+06,30,Generic brand,Mug,on-hold,1
6,4.668482e+05,10,Generic brand,Bags,shipment_delivered,1
7,1.569960e+06,45,Gildan,Sweatshirt,fulfilled,1
8,2.004430e+06,29,Gildan,Sweatshirt,shipment_delivered,1
9,2.255664e+05,25,Bella+Canvas,T-Shirt,shipment_delivered,1


In [5]:
new_line_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57583 entries, 0 to 57582
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ORDER_ID           57583 non-null  float64
 1   PRINT_PROVIDER_ID  57583 non-null  int64  
 2   PRODUCT_BRAND      57583 non-null  object 
 3   PRODUCT_TYPE       57583 non-null  object 
 4   ITEM_STATUS        57583 non-null  object 
 5   QUANTITY           57583 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 2.6+ MB


# Clean The data

## line_items cleaning

In [6]:
# Remove duplicates 
line_items = new_line_items.drop_duplicates(subset=['ORDER_ID', 'PRINT_PROVIDER_ID', 'PRODUCT_BRAND', 'PRODUCT_TYPE', 'ITEM_STATUS'])

# DOn't remove orders with similar characteristics except quantity
#line_items = new_line_items.drop_duplicates(subset=['ORDER_ID', 'PRINT_PROVIDER_ID', 'PRODUCT_BRAND', 'PRODUCT_TYPE', 'ITEM_STATUS', 'QUANTITY'])

line_items.reset_index(drop=True, inplace=True)
line_items

,ORDER_ID,PRINT_PROVIDER_ID,PRODUCT_BRAND,PRODUCT_TYPE,ITEM_STATUS,QUANTITY
0,1.750015e+06,30,Generic brand,Mug,on-hold,3
1,1.750015e+06,30,Generic brand,Mug,on-hold,2
2,1.225155e+06,16,Gildan,Kids clothes,shipment_delivered,1
3,1.828891e+06,39,Bella+Canvas,T-Shirt,shipment_delivered,1
4,1.847848e+06,10,Generic brand,Accessories,shipment_delivered,1
...,...,...,...,...,...,...
15616,1.735807e+06,2,Generic brand,Canvas,shipment_delivered,1
15617,1.732949e+06,26,Gildan,T-Shirt,fulfilled,1
15618,1.449381e+06,29,Gildan,Sweatshirt,shipment_out_for_delivery,1
15619,1.735807e+06,48,Generic brand,Mug,shipment_delivered,1


## Orders cleaning

In [7]:
orders

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,TOTAL_SHIPPING,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT
0,7143578,2.013244e+06,2013244,US,WA,2020-10-02 19:30:15,2020-10-20 07:53:06,NaN,9,2902,2200,2020-05-19 15:22:57,True,business_account_3,UPS,2020-10-08 21:07:29
1,5888534,1.358430e+06,1358429,US,MN,2020-11-21 05:19:03,2020-11-23 00:23:32,NaN,4,1149,924,2018-02-08 07:34:02,True,business_account_3,USPS,2020-11-28 21:50:56
2,6484224,9.393902e+05,939390,US,AK,2020-10-25 01:39:08,2020-10-26 20:30:53,NaN,1,977,400,2019-04-09 20:02:46,True,business_account_3,USPS,2020-11-02 23:37:54
3,7251033,1.856107e+06,1856107,US,PA,2020-10-08 16:35:21,2020-10-10 15:02:47,NaN,1,661,800,2020-07-20 17:04:41,True,business_account_3,UPS,2020-10-15 18:08:37
4,7241488,1.843904e+06,1843904,US,CO,2020-11-17 19:30:10,2020-11-18 03:45:25,NaN,2,719,424,2020-07-15 18:47:51,True,business_account_3,USPS,2020-11-24 00:44:56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13501,7562898,2.225081e+06,2225081,CA,ON,2020-11-05 21:12:29,NaT,NaN,1,0,0,2020-10-29 16:51:47,False,NaN,NaN,NaT
13502,7475289,2.153683e+06,2153683,US,CO,2020-11-16 04:11:43,NaT,NaN,1,1447,474,2020-09-30 23:11:41,True,business_account_3,NaN,NaT
13503,7144356,1.717608e+06,1717607,US,CT,2020-12-25 00:08:07,NaT,NaN,1,2187,824,2020-05-20 01:38:41,False,NaN,NaN,NaT
13504,7581194,2.243421e+06,2243420,US,IN,2020-11-19 16:43:04,NaT,NaN,4,2187,824,2020-11-05 20:02:09,False,NaN,NaN,NaT


In [8]:
# First rows encounter with duplicated order_id 
duplicate_orders1 = orders[orders['ORDER_ID'].duplicated(keep='last')]
duplicate_orders1

# Second rows encounter with duplicated order_id 
duplicate_orders2 = orders[orders['ORDER_ID'].duplicated(keep='first')]
duplicate_orders2

# Combine the duplicated rows into one dataframe for closer comparison
duplicates = pd.concat([duplicate_orders1, duplicate_orders2]).sort_values(by=['MERCHANT_ID'])
duplicates

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,TOTAL_SHIPPING,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT
537,5868230,3.635962e+05,363596,US,VI,2020-11-18 19:36:02,2020-11-19 15:23:46,NaN,2,9533,2616,2018-01-16 11:29:38,True,business_account_3,USPS,2020-11-24 17:34:42
538,5868230,3.635962e+05,363596,US,VI,2020-11-18 19:36:02,2020-11-19 15:23:46,NaN,2,9533,2616,2018-01-16 11:29:38,True,business_account_3,USPS,2020-11-28 16:51:06
5436,6118692,5.160665e+05,516066,US,MD,2020-10-11 04:30:55,2020-10-13 01:11:15,NaN,1,1979,850,2018-09-13 12:23:50,True,business_account_3,USPS,2020-10-16 19:44:40
5437,6118692,5.160665e+05,516066,US,MD,2020-10-11 04:30:55,2020-10-13 01:11:15,NaN,1,1979,850,2018-09-13 12:23:50,True,business_account_3,UPS_MAIL_INNOVATIONS,2020-10-19 19:21:56
13,6314037,7.521543e+05,752154,US,WA,2020-10-03 13:05:53,2020-10-07 20:20:59,NaN,2,14732,3884,2019-01-25 17:38:56,False,business_account_3,USPS,2020-10-09 21:38:49
14,6314037,7.521543e+05,752154,US,WA,2020-10-03 13:05:53,2020-10-07 20:20:59,NaN,2,14732,3884,2019-01-25 17:38:56,False,business_account_3,NaN,NaT
7502,7016547,1.555863e+06,1555863,US,TX,2020-10-29 22:10:53,2020-11-02 18:07:59,NaN,1,167,350,2020-02-17 23:36:26,True,business_account_3,USPS,2020-11-05 20:05:27
5285,7016547,1.555863e+06,1555863,US,LA,2020-10-11 01:33:06,2020-10-14 21:42:00,NaN,1,103,350,2020-02-17 23:36:26,True,business_account_3,NaN,NaT
6069,7223323,1.820693e+06,1820693,US,SC,2020-11-04 04:20:21,2020-11-09 18:34:04,NaN,3,6855,2624,2020-07-06 19:27:58,True,business_account_3,USPS,2020-11-09 22:02:28
6070,7223323,1.820693e+06,1820693,US,SC,2020-11-04 04:20:21,2020-11-09 18:34:04,NaN,3,6855,2624,2020-07-06 19:27:58,True,business_account_3,USPS,2020-11-13 20:28:54


In [9]:
orders.drop_duplicates(subset=['MERCHANT_ID', 'ORDER_ID'], 
                               inplace=True, keep='last')
orders

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,TOTAL_SHIPPING,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT
0,7143578,2.013244e+06,2013244,US,WA,2020-10-02 19:30:15,2020-10-20 07:53:06,NaN,9,2902,2200,2020-05-19 15:22:57,True,business_account_3,UPS,2020-10-08 21:07:29
1,5888534,1.358430e+06,1358429,US,MN,2020-11-21 05:19:03,2020-11-23 00:23:32,NaN,4,1149,924,2018-02-08 07:34:02,True,business_account_3,USPS,2020-11-28 21:50:56
2,6484224,9.393902e+05,939390,US,AK,2020-10-25 01:39:08,2020-10-26 20:30:53,NaN,1,977,400,2019-04-09 20:02:46,True,business_account_3,USPS,2020-11-02 23:37:54
3,7251033,1.856107e+06,1856107,US,PA,2020-10-08 16:35:21,2020-10-10 15:02:47,NaN,1,661,800,2020-07-20 17:04:41,True,business_account_3,UPS,2020-10-15 18:08:37
4,7241488,1.843904e+06,1843904,US,CO,2020-11-17 19:30:10,2020-11-18 03:45:25,NaN,2,719,424,2020-07-15 18:47:51,True,business_account_3,USPS,2020-11-24 00:44:56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13501,7562898,2.225081e+06,2225081,CA,ON,2020-11-05 21:12:29,NaT,NaN,1,0,0,2020-10-29 16:51:47,False,NaN,NaN,NaT
13502,7475289,2.153683e+06,2153683,US,CO,2020-11-16 04:11:43,NaT,NaN,1,1447,474,2020-09-30 23:11:41,True,business_account_3,NaN,NaT
13503,7144356,1.717608e+06,1717607,US,CT,2020-12-25 00:08:07,NaT,NaN,1,2187,824,2020-05-20 01:38:41,False,NaN,NaN,NaT
13504,7581194,2.243421e+06,2243420,US,IN,2020-11-19 16:43:04,NaT,NaN,4,2187,824,2020-11-05 20:02:09,False,NaN,NaN,NaT


In [10]:
# Not complete in removing the errors
orders.drop_duplicates(subset=['MERCHANT_ID', 'ORDER_ID', 
                               'SHOP_ID', 'ADDRESS_TO_COUNTRY', 
                               'ADDRESS_TO_REGION','ORDER_DT',
                               'FULFILLED_DT','REPRINT_FLAG',
                               'SALES_CHANNEL_TYPE_ID',
                               'TOTAL_COST','TOTAL_SHIPPING',
                               'MERCHANT_REGISTERED_DT',
                               'SUB_IS_ACTIVE_FLAG','SUB_PLAN', 
                               'SHIPMENT_CARRIER'], inplace=True,
                                  keep='last')
orders.reset_index(drop=True, inplace=True)
orders

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,TOTAL_SHIPPING,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT
0,7143578,2.013244e+06,2013244,US,WA,2020-10-02 19:30:15,2020-10-20 07:53:06,NaN,9,2902,2200,2020-05-19 15:22:57,True,business_account_3,UPS,2020-10-08 21:07:29
1,5888534,1.358430e+06,1358429,US,MN,2020-11-21 05:19:03,2020-11-23 00:23:32,NaN,4,1149,924,2018-02-08 07:34:02,True,business_account_3,USPS,2020-11-28 21:50:56
2,6484224,9.393902e+05,939390,US,AK,2020-10-25 01:39:08,2020-10-26 20:30:53,NaN,1,977,400,2019-04-09 20:02:46,True,business_account_3,USPS,2020-11-02 23:37:54
3,7251033,1.856107e+06,1856107,US,PA,2020-10-08 16:35:21,2020-10-10 15:02:47,NaN,1,661,800,2020-07-20 17:04:41,True,business_account_3,UPS,2020-10-15 18:08:37
4,7241488,1.843904e+06,1843904,US,CO,2020-11-17 19:30:10,2020-11-18 03:45:25,NaN,2,719,424,2020-07-15 18:47:51,True,business_account_3,USPS,2020-11-24 00:44:56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13490,7562898,2.225081e+06,2225081,CA,ON,2020-11-05 21:12:29,NaT,NaN,1,0,0,2020-10-29 16:51:47,False,NaN,NaN,NaT
13491,7475289,2.153683e+06,2153683,US,CO,2020-11-16 04:11:43,NaT,NaN,1,1447,474,2020-09-30 23:11:41,True,business_account_3,NaN,NaT
13492,7144356,1.717608e+06,1717607,US,CT,2020-12-25 00:08:07,NaT,NaN,1,2187,824,2020-05-20 01:38:41,False,NaN,NaN,NaT
13493,7581194,2.243421e+06,2243420,US,IN,2020-11-19 16:43:04,NaT,NaN,4,2187,824,2020-11-05 20:02:09,False,NaN,NaN,NaT


In [11]:
orders

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,TOTAL_SHIPPING,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT
0,7143578,2.013244e+06,2013244,US,WA,2020-10-02 19:30:15,2020-10-20 07:53:06,NaN,9,2902,2200,2020-05-19 15:22:57,True,business_account_3,UPS,2020-10-08 21:07:29
1,5888534,1.358430e+06,1358429,US,MN,2020-11-21 05:19:03,2020-11-23 00:23:32,NaN,4,1149,924,2018-02-08 07:34:02,True,business_account_3,USPS,2020-11-28 21:50:56
2,6484224,9.393902e+05,939390,US,AK,2020-10-25 01:39:08,2020-10-26 20:30:53,NaN,1,977,400,2019-04-09 20:02:46,True,business_account_3,USPS,2020-11-02 23:37:54
3,7251033,1.856107e+06,1856107,US,PA,2020-10-08 16:35:21,2020-10-10 15:02:47,NaN,1,661,800,2020-07-20 17:04:41,True,business_account_3,UPS,2020-10-15 18:08:37
4,7241488,1.843904e+06,1843904,US,CO,2020-11-17 19:30:10,2020-11-18 03:45:25,NaN,2,719,424,2020-07-15 18:47:51,True,business_account_3,USPS,2020-11-24 00:44:56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13490,7562898,2.225081e+06,2225081,CA,ON,2020-11-05 21:12:29,NaT,NaN,1,0,0,2020-10-29 16:51:47,False,NaN,NaN,NaT
13491,7475289,2.153683e+06,2153683,US,CO,2020-11-16 04:11:43,NaT,NaN,1,1447,474,2020-09-30 23:11:41,True,business_account_3,NaN,NaT
13492,7144356,1.717608e+06,1717607,US,CT,2020-12-25 00:08:07,NaT,NaN,1,2187,824,2020-05-20 01:38:41,False,NaN,NaN,NaT
13493,7581194,2.243421e+06,2243420,US,IN,2020-11-19 16:43:04,NaT,NaN,4,2187,824,2020-11-05 20:02:09,False,NaN,NaN,NaT


## Prepare for Joining

In [12]:
line_items

,ORDER_ID,PRINT_PROVIDER_ID,PRODUCT_BRAND,PRODUCT_TYPE,ITEM_STATUS,QUANTITY
0,1.750015e+06,30,Generic brand,Mug,on-hold,3
1,1.750015e+06,30,Generic brand,Mug,on-hold,2
2,1.225155e+06,16,Gildan,Kids clothes,shipment_delivered,1
3,1.828891e+06,39,Bella+Canvas,T-Shirt,shipment_delivered,1
4,1.847848e+06,10,Generic brand,Accessories,shipment_delivered,1
...,...,...,...,...,...,...
15616,1.735807e+06,2,Generic brand,Canvas,shipment_delivered,1
15617,1.732949e+06,26,Gildan,T-Shirt,fulfilled,1
15618,1.449381e+06,29,Gildan,Sweatshirt,shipment_out_for_delivery,1
15619,1.735807e+06,48,Generic brand,Mug,shipment_delivered,1


In [13]:
pd.options.mode.chained_assignment = None  # default='warn'
for i in range(len(line_items['ORDER_ID'])):
    order = str(line_items.loc[i]['ORDER_ID']).split(".")
    line_items.at[i, 'ORDER_ID'] = int(order[-1])
    line_items.at[i,'SHOP_ID'] = int(order[0])
line_items

,ORDER_ID,PRINT_PROVIDER_ID,PRODUCT_BRAND,PRODUCT_TYPE,ITEM_STATUS,QUANTITY,SHOP_ID
0,15208.0,30,Generic brand,Mug,on-hold,3,1750015.0
1,12357.0,30,Generic brand,Mug,on-hold,2,1750015.0
2,10081.0,16,Gildan,Kids clothes,shipment_delivered,1,1225155.0
3,707.0,39,Bella+Canvas,T-Shirt,shipment_delivered,1,1828890.0
4,55.0,10,Generic brand,Accessories,shipment_delivered,1,1847847.0
...,...,...,...,...,...,...,...
15616,1367.0,2,Generic brand,Canvas,shipment_delivered,1,1735807.0
15617,7852.0,26,Gildan,T-Shirt,fulfilled,1,1732948.0
15618,11854.0,29,Gildan,Sweatshirt,shipment_out_for_delivery,1,1449381.0
15619,2759.0,48,Generic brand,Mug,shipment_delivered,1,1735807.0


In [14]:
for i in range(len(orders['ORDER_ID'])):
    order = str(orders.loc[i]['ORDER_ID']).split(".")
    orders.at[i, 'ORDER_ID'] = int(order[-1])
orders

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,TOTAL_SHIPPING,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT
0,7143578,178.0,2013244,US,WA,2020-10-02 19:30:15,2020-10-20 07:53:06,NaN,9,2902,2200,2020-05-19 15:22:57,True,business_account_3,UPS,2020-10-08 21:07:29
1,5888534,695.0,1358429,US,MN,2020-11-21 05:19:03,2020-11-23 00:23:32,NaN,4,1149,924,2018-02-08 07:34:02,True,business_account_3,USPS,2020-11-28 21:50:56
2,6484224,193.0,939390,US,AK,2020-10-25 01:39:08,2020-10-26 20:30:53,NaN,1,977,400,2019-04-09 20:02:46,True,business_account_3,USPS,2020-11-02 23:37:54
3,7251033,4.0,1856107,US,PA,2020-10-08 16:35:21,2020-10-10 15:02:47,NaN,1,661,800,2020-07-20 17:04:41,True,business_account_3,UPS,2020-10-15 18:08:37
4,7241488,4.0,1843904,US,CO,2020-11-17 19:30:10,2020-11-18 03:45:25,NaN,2,719,424,2020-07-15 18:47:51,True,business_account_3,USPS,2020-11-24 00:44:56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13490,7562898,12.0,2225081,CA,ON,2020-11-05 21:12:29,NaT,NaN,1,0,0,2020-10-29 16:51:47,False,NaN,NaN,NaT
13491,7475289,2224.0,2153683,US,CO,2020-11-16 04:11:43,NaT,NaN,1,1447,474,2020-09-30 23:11:41,True,business_account_3,NaN,NaT
13492,7144356,5.0,1717607,US,CT,2020-12-25 00:08:07,NaT,NaN,1,2187,824,2020-05-20 01:38:41,False,NaN,NaN,NaT
13493,7581194,66.0,2243420,US,IN,2020-11-19 16:43:04,NaT,NaN,4,2187,824,2020-11-05 20:02:09,False,NaN,NaN,NaT


# Join the Data

In [15]:
# Join the two dataframes to form one where each merchant is 
# Matched with all their orders and shops
printify_df = pd.merge(orders, line_items, how="inner", on=['ORDER_ID','SHOP_ID'])

In [16]:
printify_df

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,...,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT,PRINT_PROVIDER_ID,PRODUCT_BRAND,PRODUCT_TYPE,ITEM_STATUS,QUANTITY
0,7143578,178.0,2013244,US,WA,2020-10-02 19:30:15,2020-10-20 07:53:06,NaN,9,2902,...,2020-05-19 15:22:57,True,business_account_3,UPS,2020-10-08 21:07:29,1,Generic brand,Mug,shipment_delivered,1
1,7143578,178.0,2013244,US,WA,2020-10-02 19:30:15,2020-10-20 07:53:06,NaN,9,2902,...,2020-05-19 15:22:57,True,business_account_3,UPS,2020-10-08 21:07:29,16,Generic brand,Mug,shipment_delivered,1
2,5888534,695.0,1358429,US,MN,2020-11-21 05:19:03,2020-11-23 00:23:32,NaN,4,1149,...,2018-02-08 07:34:02,True,business_account_3,USPS,2020-11-28 21:50:56,39,Gildan,Sweatshirt,shipment_delivered,1
3,6484224,193.0,939390,US,AK,2020-10-25 01:39:08,2020-10-26 20:30:53,NaN,1,977,...,2019-04-09 20:02:46,True,business_account_3,USPS,2020-11-02 23:37:54,25,Delta,T-Shirt,shipment_delivered,1
4,7251033,4.0,1856107,US,PA,2020-10-08 16:35:21,2020-10-10 15:02:47,NaN,1,661,...,2020-07-20 17:04:41,True,business_account_3,UPS,2020-10-15 18:08:37,16,Generic brand,Mug,shipment_delivered,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15616,7562898,12.0,2225081,CA,ON,2020-11-05 21:12:29,NaT,NaN,1,0,...,2020-10-29 16:51:47,False,NaN,NaN,NaT,10,Generic brand,Accessories,canceled,10
15617,7475289,2224.0,2153683,US,CO,2020-11-16 04:11:43,NaT,NaN,1,1447,...,2020-09-30 23:11:41,True,business_account_3,NaN,NaT,16,Sport-Tek,Hats,on-hold,1
15618,7144356,5.0,1717607,US,CT,2020-12-25 00:08:07,NaT,NaN,1,2187,...,2020-05-20 01:38:41,False,NaN,NaN,NaT,3,Gildan,Hoodie,on-hold,1
15619,7581194,66.0,2243420,US,IN,2020-11-19 16:43:04,NaT,NaN,4,2187,...,2020-11-05 20:02:09,False,NaN,NaN,NaT,3,Gildan,Hoodie,canceled,1


In [17]:
printify_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15621 entries, 0 to 15620
Data columns (total 21 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   MERCHANT_ID             15621 non-null  int64         
 1   ORDER_ID                15621 non-null  float64       
 2   SHOP_ID                 15621 non-null  int64         
 3   ADDRESS_TO_COUNTRY      15620 non-null  object        
 4   ADDRESS_TO_REGION       15009 non-null  object        
 5   ORDER_DT                15621 non-null  datetime64[ns]
 6   FULFILLED_DT            14651 non-null  datetime64[ns]
 7   REPRINT_FLAG            108 non-null    float64       
 8   SALES_CHANNEL_TYPE_ID   15621 non-null  int64         
 9   TOTAL_COST              15621 non-null  int64         
 10  TOTAL_SHIPPING          15621 non-null  int64         
 11  MERCHANT_REGISTERED_DT  15621 non-null  datetime64[ns]
 12  SUB_IS_ACTIVE_FLAG      15621 non-null  bool  

# Write a query returning total sales, orders, and count of merchants by month


In [18]:
# Add the month attribute on the dataset to make querying possible
printify_df['ORDER_DT'] = pd.to_datetime(printify_df['ORDER_DT'])
printify_df['Month'] = printify_df['ORDER_DT'].dt.month_name()

In [19]:
# Take the sum of the order quantity and total cost for each unique month
monthly_data = printify_df[['QUANTITY', 'Month', 'TOTAL_COST']].groupby(['Month']).sum()

# Group again using the month and this time get the aggregated sums
monthly_data['Merchants_count'] = printify_df[['MERCHANT_ID', 'Month']].groupby(['Month']).count()

monthly_data

,QUANTITY,TOTAL_COST,Merchants_count
Month,,,
December,6695,10991332,6168
November,6054,10742810,5578
October,4168,9132848,3875


# merchants total sales, product count, and order count ordered by order count for merchants with more than 5 orders

### Assumptions
- Total sales: the amount of money that is collected for the products
- Products: These are the specifics of an order given in the "Quantity" column
- Orders: are the collections of products that a merchant sells from a shop through sales channel given in the ORDER_ID column

In [20]:
# Create new df that has all the sums required attributes grouped merchant id
sql_quiz_2 = printify_df[['MERCHANT_ID', 'QUANTITY','TOTAL_COST']].groupby(['MERCHANT_ID']).sum()

# In the new df add all the counts attributes required grouped merchant id
sql_quiz_2['Order_count'] = printify_df[['MERCHANT_ID', 'ORDER_ID']].groupby(['MERCHANT_ID']).count()

# Rename columns then sort
sql_quiz_2.rename(columns={'QUANTITY':'Product_count', 'TOTAL_COST':'Total_sales'}, inplace=True)
sql_quiz_2 = sql_quiz_2[sql_quiz_2['Order_count'] > 5].sort_values(by=['Order_count'], ascending=False)
sql_quiz_2

,Product_count,Total_sales,Order_count
MERCHANT_ID,,,
5755049,179,527458,164
7019548,156,102547,150
6433350,119,196167,114
7027744,103,215563,102
7084134,103,82879,99
...,...,...,...
7293763,6,6965,6
7008352,15,16061,6
7019107,6,6084,6


## Write a query returning all ORDER_IDs with the time the merchant has been active at the time of the order, the rank of the merchant by order count for the previous month, and the merchant's primary sales channel for the previous month

### All ORDER_IDs with the time the merchant has been active at the time of the order

In [21]:
printify_df['Month'].unique()

array(['October', 'November', 'December'], dtype=object)

In [22]:
# Introduce a new attribute to show when a merchant has been on printify when the order was made
printify_df["Have_been_active"] = printify_df["ORDER_DT"] - printify_df["MERCHANT_REGISTERED_DT"]

# Select needed attributes
new_df = printify_df[['MERCHANT_ID', 'Have_been_active','ORDER_ID']]

# Bo need to group as the dataset was unique for the products in orders
sql_quiz_3 = new_df[['MERCHANT_ID','ORDER_ID','Have_been_active']]

sql_quiz_3

,MERCHANT_ID,ORDER_ID,Have_been_active
0,7143578,178.0,136 days 04:07:18
1,7143578,178.0,136 days 04:07:18
2,5888534,695.0,1016 days 21:45:01
3,6484224,193.0,564 days 05:36:22
4,7251033,4.0,79 days 23:30:40
...,...,...,...
15616,7562898,12.0,7 days 04:20:42
15617,7475289,2224.0,46 days 05:00:02
15618,7144356,5.0,218 days 22:29:26
15619,7581194,66.0,13 days 20:40:55


### The rank of the merchant by order count for the previous month
- Assumption previous month is November since the data last month was December

In [23]:
previous_month = 'November'

# Take a part of the data that contain all the attributes needed for the query
merchant_order_time = printify_df[['MERCHANT_ID', 'ORDER_DT','ORDER_ID']]

# Select orders only ordered in November
Previous_month_order_count= merchant_order_time[merchant_order_time['ORDER_DT'].dt.month_name() == previous_month]

# Group the data using merchant id
Previous_month_order_count[['MERCHANT_ID', 'ORDER_ID']].groupby(['MERCHANT_ID']).count()

# Drop the date column since it is not needed in the query result
Previous_month_order_count.drop(columns='ORDER_DT', inplace=True)

# Rename accordingly
Previous_month_order_count.rename(columns={'ORDER_ID':'Orders_Count', 'MERCHANT_ID':'Previous_Month_Merchants'}, inplace=True)

Previous_month_order_count

,Previous_Month_Merchants,Orders_Count
2,5888534,695.0
5,7241488,4.0
7,7146390,14704.0
9,7110135,309.0
10,5900409,102.0
...,...,...
15614,7398841,708.0
15616,7562898,12.0
15617,7475289,2224.0
15619,7581194,66.0


### The merchant's primary sales channel for the previous month

In [24]:
previous_month = 'November'

# Take a part of the data that contain all the attributes needed for the query
merchant_channel = printify_df[['MERCHANT_ID', 'SALES_CHANNEL_TYPE_ID', 'ORDER_DT']]

# Select orders only ordered in November
merchant_pri_channel = merchant_channel[merchant_channel['ORDER_DT'].dt.month_name() == previous_month]

# Group the data using merchant id
merchant_pri_channel[['MERCHANT_ID', 'SALES_CHANNEL_TYPE_ID']].groupby(['MERCHANT_ID']).agg(pd.Series.mode)

# Rename accordingly
merchant_pri_channel.rename(columns={'SALES_CHANNEL_TYPE_ID':'Primary_Sales_Channel', 'MERCHANT_ID':'Previous_Month_Merchants'}, inplace=True)

# Drop the date column since it is not needed in the query result
# merchant_pri_channel.drop(columns='ORDER_DT', inplace=True)


merchant_pri_channel

,Previous_Month_Merchants,Primary_Sales_Channel,ORDER_DT
2,5888534,4,2020-11-21 05:19:03
5,7241488,2,2020-11-17 19:30:10
7,7146390,1,2020-11-13 22:00:23
9,7110135,4,2020-11-13 04:55:57
10,5900409,1,2020-11-11 11:19:05
...,...,...,...
15614,7398841,6,2020-11-06 08:54:08
15616,7562898,1,2020-11-05 21:12:29
15617,7475289,1,2020-11-16 04:11:43
15619,7581194,4,2020-11-19 16:43:04
